In [1]:
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt

datananme = 'js'

In [2]:
# read the data using pd

# pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 200)

# the data is stored in the parent dir with dir_name being datananme
train_pd = pd.read_parquet(f'../{datananme}/train.parquet/')
train_pd.shape

(47127338, 93)

In [ ]:
# read the data using pl

train_pl = pl.read_parquet(f'../{datananme}/train.parquet/')
train_pl.shape

### Data checking/visualizing

In [ ]:
# missing values for all of the features for rows that 'responder_6'
# is not None

from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter

supervised_usable = (
    train_pl
    .filter(pl.col('responder_6').is_not_null())
)

missing_count = (
    supervised_usable
    .null_count()
    .transpose(include_header=True,
               header_name='feature',
               column_names=['null_count'])
    .sort('null_count', descending=True)
    .with_columns((pl.col('null_count') / len(supervised_usable)).alias('null_ratio'))
)

plt.figure(figsize=(6, 20))
plt.title(f'Missing values over the {len(supervised_usable)} samples which have a target')
plt.barh(np.arange(len(missing_count)), missing_count.get_column('null_ratio'), color='coral', label='missing')
plt.barh(np.arange(len(missing_count)), 
         1 - missing_count.get_column('null_ratio'),
         left=missing_count.get_column('null_ratio'),
         color='darkseagreen', label='available')
plt.yticks(np.arange(len(missing_count)), missing_count.get_column('feature'))
plt.gca().xaxis.set_major_formatter(PercentFormatter(xmax=1, decimals=0))
plt.xlim(0, 1)
plt.legend()
plt.show()

In [ ]:
# check NaNs for 'responder_6'
train_pd['responder_6'].isna().sum()

In [ ]:
# view lags.parquet

lags_pd = pd.read_parquet(f'../{datananme}/lags.parquet/date_id=0')
lags_pd

In [ ]:
# view the sample test data

test_pd = pd.read_parquet(f'../{datananme}/test.parquet/date_id=0/part-0.parquet')
test_pd

### Data pre-processing

In [3]:
# drop some features

col_to_drop = [f'responder_{i}' for i in range(9) if i != 6] \
                + ['partition_id']

train_pd1 = train_pd.drop(labels=col_to_drop, axis=1)

In [4]:
# drop rows with NAs
# but perhaps no need? xgboost shall be able to handle NA by default

train_pd2 = train_pd1.dropna(axis=0)
print(train_pd2.shape)

(35370822, 84)


In [5]:
# split to predictors and y, since there is no categorical features

col_weight = ['weight']
col_y = ['responder_6']
col_num = [col for col in train_pd2.columns.tolist() if col not in col_y]

### Model training

In [ ]:
# def calculate_r2(y_true, y_pred, weights):
#     """
#     Args:
#         y_true, y_pred, weights: np.arrays.
#     """
#     numerator = np.sum(weights * (y_true - y_pred) ** 2)
#     denominator = np.sum(weights * (y_true ** 2))
#     r2_score = 1 - (numerator / denominator)
#     return r2_score

In [ ]:
def calculate_r2_batched(y_true, y_pred, weights, batch_size=100000):
    """
    Calculate the weighted R^2 score using small batches.
    
    Args:
        y_true, y_pred, weights: np.arrays.
    """
    total_weighted_error = 0
    total_weighted_square_true = 0
    num_batches = int(np.ceil(len(y_true) / batch_size))

    for i in range(num_batches):
        start = i * batch_size
        end = min((i + 1) * batch_size, len(y_true))

        y_true_batch = y_true[start:end]
        y_pred_batch = y_pred[start:end]
        weights_batch = weights[start:end]

        numerator_batch = np.sum(weights_batch * (y_true_batch - y_pred_batch) ** 2)
        denominator_batch = np.sum(weights_batch * y_true_batch ** 2)

        total_weighted_error += numerator_batch
        total_weighted_square_true += denominator_batch
        
        print('-', end='') # TEST

    r2_score = 1 - (total_weighted_error / total_weighted_square_true)
    return r2_score

r2_score = calculate_r2_batched(y_valid.to_numpy(), y_valid_pred, w_valid.to_numpy())
r2_score

-----------------------------------------------

In [ ]:
def time_series_cross_validation(date_ids, n_splits):
    fold_size = len(date_ids) // n_splits

    for i in range(n_splits):
        # Here, the training set includes all data up to i-th fold
        end_train = (i + 1) * fold_size
        train_dates = date_ids[:end_train]
        # Validation dates are the next fold_size data points after the training set
        valid_dates = date_ids[end_train:end_train + fold_size] if end_train + fold_size <= len(date_ids) else date_ids[end_train:]
        
        yield train_dates, valid_dates
        

# date_ids = train_pd2['date_id'].unique()

# n_splits = 5

# for train_dates, valid_dates in time_series_cross_validation(date_ids, n_splits):
#     print("Train Dates:", train_dates)
#     print("Valid Dates:", valid_dates)
#     print()

In [17]:
# xgboost
# cross-validation on dates
import xgboost as xgb

## some hyperparameters 
verbose_eval = 10
early_stopping_rounds = 100
num_boost_round = 1000

## main part
date_ids = train_pd2['date_id'].unique()
k = 5
fold_size = date_ids.shape[0] // k

# for i in range(k):
#     start = i * fold_size
#     end = (i + 1) * fold_size if i != k - 1 else len(date_ids_ls)
#     valid_dates = date_ids[start:end]
#     train_dates = np.concatenate([date_ids[:start], date_ids[end:]])

for train_dates, valid_dates in time_series_cross_validation(date_ids, n_splits):
    X_train = train_pd2[col_num].loc[train_pd2['date_id'].isin(train_dates)]
    y_train = train_pd2[col_y].loc[train_pd2['date_id'].isin(train_dates)]
    w_train = train_pd2[col_weight].loc[train_pd2['date_id'].isin(train_dates)]

    X_valid = train_pd2[col_num].loc[train_pd2['date_id'].isin(valid_dates)]
    y_valid = train_pd2[col_y].loc[train_pd2['date_id'].isin(valid_dates)]
    w_valid = train_pd2[col_weight].loc[train_pd2['date_id'].isin(valid_dates)]

    dtrain = xgb.DMatrix(X_train, label=y_train, weight=w_train)
    dvalid = xgb.DMatrix(X_valid, label=y_valid, weight=w_valid)

    XGB_PARAMS = {
        'eval_metric': 'rmse',
        'learning_rate': 0.5,
        'max_depth': 12,
        'min_child_weight': 1.5,
        'subsample': 0.8555,
        'colsample_bytree': 0.85555555,
        'random_state': 42,
        'tree_method': 'hist',
        'device': 'cuda'
    }

    model = xgb.train(XGB_PARAMS, dtrain, num_boost_round=num_boost_round, evals=[(dtrain, 'train'), (dvalid, 'valid')], early_stopping_rounds=early_stopping_rounds, verbose_eval=verbose_eval)

    y_valid_pred = model.predict(dvalid)
#     r2_score = calculate_r2_batched(y_valid.to_numpy(), y_valid_pred, w_valid.to_numpy())
#     print(f"Fold {fold_idx} validation R2 score: {r2_score}")
    print('\nFold completed \n\n')

[0]	train-rmse:0.80886	valid-rmse:0.85211
[10]	train-rmse:0.74917	valid-rmse:0.91122
[20]	train-rmse:0.69859	valid-rmse:0.98388
[30]	train-rmse:0.66358	valid-rmse:1.02927
[40]	train-rmse:0.63271	valid-rmse:1.06170
[50]	train-rmse:0.60450	valid-rmse:1.09480
[60]	train-rmse:0.57909	valid-rmse:1.12086
[70]	train-rmse:0.55682	valid-rmse:1.13795
[80]	train-rmse:0.53793	valid-rmse:1.15427
[90]	train-rmse:0.52049	valid-rmse:1.16452
[99]	train-rmse:0.50372	valid-rmse:1.18246
[0]	train-rmse:0.82528	valid-rmse:0.82750
[10]	train-rmse:0.77882	valid-rmse:0.85390
[20]	train-rmse:0.74762	valid-rmse:0.90764
[30]	train-rmse:0.72051	valid-rmse:0.92927
[40]	train-rmse:0.69606	valid-rmse:0.95881
[50]	train-rmse:0.67815	valid-rmse:0.98269
[60]	train-rmse:0.65932	valid-rmse:1.00400
[70]	train-rmse:0.64268	valid-rmse:1.04444
[80]	train-rmse:0.62631	valid-rmse:1.05778
[90]	train-rmse:0.61182	valid-rmse:1.08960
[99]	train-rmse:0.59673	valid-rmse:1.10067
[0]	train-rmse:0.82289	valid-rmse:0.76226
[10]	train-rms

XGBoostError: [16:03:39] /workspace/src/c_api/../data/../common/device_helpers.cuh:431: Memory allocation error on worker 0: std::bad_alloc: cudaErrorMemoryAllocation: out of memory
- Free memory: 4877320192
- Requested memory: 5237352456

Stack trace:
  [bt] (0) /home/cai00196/.conda/envs/js/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x77f79a) [0x7f8a0aed379a]
  [bt] (1) /home/cai00196/.conda/envs/js/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x783994) [0x7f8a0aed7994]
  [bt] (2) /home/cai00196/.conda/envs/js/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x79d5ca) [0x7f8a0aef15ca]
  [bt] (3) /home/cai00196/.conda/envs/js/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x7d1df4) [0x7f8a0af25df4]
  [bt] (4) /home/cai00196/.conda/envs/js/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x7d1f44) [0x7f8a0af25f44]
  [bt] (5) /home/cai00196/.conda/envs/js/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0xad5fa7) [0x7f8a0b229fa7]
  [

In [ ]:
# test xgboost
# import xgboost as xgb
# import numpy as np

# # Generate some random data to simulate a training set
# num_samples = 1000
# num_features = 10

# X = np.random.rand(num_samples, num_features)
# y = np.random.randint(2, size=num_samples)

# # Split data into training and evaluation sets
# split_index = int(num_samples * 0.8)
# X_train, X_eval = X[:split_index], X[split_index:]
# y_train, y_eval = y[:split_index], y[split_index:]

# # Create DMatrix for train and eval data
# dtrain = xgb.DMatrix(X_train, label=y_train)
# deval = xgb.DMatrix(X_eval, label=y_eval)

# # Define the parameter dictionary, enabling the use of GPU
# param = {
#     'objective': 'binary:logistic',
#     'max_depth': 3,
#     'eta': 0.1,
#     'tree_method': 'hist',  # Specify 'gpu_hist' to enable GPU support
#     'eval_metric': 'logloss',
#     'device': 'cuda:4'
# }

# # Number of boosting rounds
# num_round = 10

# # Create a watchlist to evaluate the performance of the model
# evals = [(dtrain, 'train'), (deval, 'eval')]

# # Train the model
# bst = xgb.train(param, dtrain, num_round, evals)

# # Print model information
# print(bst)

#### Evaluation 